In [16]:
import json
import pandas as pd
from tqdm import tqdm

In [2]:
with open('../../examples/relevant_jobs_classification/data/test_converted.json','r') as f:
    data = json.load(f)

In [3]:
# data

In [44]:
df =pd.read_csv('../../examples/relevant_jobs_classification/data/analysis_dataset.csv')

In [45]:
df

,job_id,application_id,user_id
0,1373681,144274567,439042
1,1373681,144261479,2015063
2,1373681,144253966,2777927
3,1373681,144241259,58130
4,1373681,144236812,2290933
...,...,...,...
136248,1376058,144442227,2151474
136249,1375030,144442249,1463255
136250,1373682,144442251,2601930
136251,1376582,144442257,1417492


In [60]:
df['user_id']=df['user_id'].astype(int)

In [61]:
df.dtypes

job_id            int64
application_id    int64
user_id           int64
dtype: object

In [8]:
uid_list = list(data.keys())[:5]

In [42]:
# data

In [71]:
df_list = []

for id, desc in tqdm(data.items()):
    try:
        descriptions = "; ".join([exp['description'] for exp in desc])
        df_list.append({'id': id, 'descriptions': descriptions})
    except Exception as e:
        # print(e)
        continue

udf = pd.DataFrame(df_list)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 389/389 [00:00<00:00, 575514.73it/s]


In [72]:
udf

,id,descriptions
0,209995,Initiating Coverage and tracking companies wit...
1,882096,Tracking Banking sector with 25 stocks uneer a...
2,761462,Part of 4-member investment team of a sector a...
3,2749339,Independently raised INR 10 Cr for early-stage...
4,1227,Setting up the Family office / Treasury Head /...
...,...,...
302,2700303,Preparing Job descriptions in consultation wit...
303,1968051,Continuous monitoring and preparation of COP w...
304,2754531,Zero Based Budgeting and Overhead control- Act...
305,2753323,Oversaw sales and marketing in entire North In...


In [73]:
udf['id']=udf['id'].astype(int)

In [74]:
# df[1:2].descriptions.values

In [75]:
# data['2673538']

In [76]:
udf.dtypes

id               int64
descriptions    object
dtype: object

In [77]:
udlist=list(udf.id)

In [78]:
len(udlist)

307

In [79]:
df.loc[df['user_id'].isin(udlist)].user_id.nunique()

307

In [80]:
df.loc[df['user_id'].isin(udlist)]

,job_id,application_id,user_id
47,1373682,144186487,831985
51,1373682,144174113,36956
63,1373682,144149169,2432230
68,1373683,144193247,397888
80,1373684,144282598,218127
...,...,...,...
135895,1376077,144441645,835564
135901,1376085,144441636,835564
135910,1376037,144441658,835564
135912,1375991,144441661,835564


In [81]:
df.user_id.nunique()

30363